In [22]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [23]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-05 17:48:57.148867


In [24]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [25]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [26]:
current_date.to_datetime64()

numpy.datetime64('2025-03-05T17:00:00.000000000')

In [27]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=30)

In [28]:
fetch_data_to

Timestamp('2025-03-05 17:00:00+0000', tz='UTC')

In [29]:
fetch_data_from

Timestamp('2025-02-03 17:00:00+0000', tz='UTC')

In [30]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks and remove timezone info
    historical_from_date = (from_date - timedelta(weeks=52)).replace(tzinfo=None)
    historical_to_date = (to_date - timedelta(weeks=52)).replace(tzinfo=None)

    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from['pickup_datetime'] = rides_from['pickup_datetime'].dt.tz_localize(None)
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to['pickup_datetime'] = rides_to['pickup_datetime'].dt.tz_localize(None)
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    rides['pickup_datetime'] += timedelta(weeks=52)
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [31]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...


Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [32]:
rides

,pickup_datetime,pickup_location_id
843972,2025-02-12 16:25:44,2
1150673,2025-02-15 16:56:40,2
12684,2025-02-03 18:07:44,3
34388,2025-02-04 04:14:50,3
35989,2025-02-04 06:08:40,3
...,...,...
3042498,2025-03-05 16:58:50,263
3044586,2025-03-05 16:58:54,263
3038037,2025-03-05 16:59:00,263
3044149,2025-03-05 16:59:01,263


In [33]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [34]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-03 17:00:00,2,0
1,2025-02-03 18:00:00,2,0
2,2025-02-03 19:00:00,2,0
3,2025-02-03 20:00:00,2,0
4,2025-02-03 21:00:00,2,0
...,...,...,...
181435,2025-03-05 12:00:00,263,100
181436,2025-03-05 13:00:00,263,108
181437,2025-03-05 14:00:00,263,136
181438,2025-03-05 15:00:00,263,153


In [35]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181440 entries, 0 to 181439
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         181440 non-null  datetime64[ns]
 1   pickup_location_id  181440 non-null  int16         
 2   rides               181440 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.1 MB


In [36]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-03-05 12:49:01,749 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 12:49:01,756 INFO: Initializing external client
2025-03-05 12:49:01,756 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 12:49:02,277 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214681


In [37]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 181440/181440 | Elapsed Time: 00:12 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/taxi_project/Resources/jobs/time_series_hourly_feature_group_1_offline_fg_materialization/config_1741196770165) to trigger the materialization job again.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)